# __Poisson matrix tests__

## Outline
- __Dependencies__
- __Dynamic ansatz__
    - Noiseless
    - With noise using Qiskit AER
- __Static ansatz__
    - Noiseless
    - With noise using Qiskit AER

In [ ]:
import numpy as np
from VariationalLinearQuantumSolver.device_var_lse_solver import DeviceVarLSESolver
from VariationalLinearQuantumSolver.device import DeviceType
from VariationalLinearQuantumSolver.conditioned_matrix import poissonMatrix
from VariationalLinearQuantumSolver.non_dynamical_ansatz import fixed_layered_ansatz
from tqdm import tqdm
from VariationalLinearQuantumSolver.device import Device

qubits = 4
b = np.ones(2**qubits)/np.sqrt(2**qubits)
depth=10

assert qubits % 2 == 0 # Needed for Poisson matrix

In [5]:
# As defined by [10]
def TRC_ADA(iteration_count : dict):
    trc = 0
    for d in iteration_count:
        z = iteration_count.get(d)
        trc += d*z
    return trc

def TRC_ASA(iteration_count : dict):
    trc = 0
    for d in iteration_count:
        z = iteration_count.get(d)
        trc += z*depth
    return trc

## __Dynamic ansatz__

In [6]:
# For poisson matrix
N_runs = 1#10
N_steps = 2#1500
redo_calc = 2#5

local_noiseless_dynamic_trc = np.zeros((redo_calc, ))
global_noiseless_dynamic_trc = np.zeros((redo_calc, ))
local_noise_dynamic_trc = np.zeros((redo_calc, ))
global_noise_dynamic_trc = np.zeros((redo_calc, ))

local_noiseless_static_trc = np.zeros((redo_calc, ))
global_noiseless_static_trc = np.zeros((redo_calc, ))
local_noise_static_trc = np.zeros((redo_calc, ))
global_noise_static_trc = np.zeros((redo_calc, ))

local_noise_dynamic_error = np.zeros((redo_calc, ))
local_noiseless_dynamic_error = np.zeros((redo_calc, ))
local_noise_static_error = np.zeros((redo_calc, ))
local_noiseless_static_error = np.zeros((redo_calc, ))

global_noise_dynamic_error = np.zeros((redo_calc, ))
global_noiseless_dynamic_error = np.zeros((redo_calc, ))
global_noise_static_error = np.zeros((redo_calc, ))
global_noiseless_static_error = np.zeros((redo_calc, ))

local_noise_dynamic_solution = np.zeros((redo_calc, 2**qubits))
local_noiseless_dynamic_solution = np.zeros((redo_calc, 2**qubits))
local_noise_static_solution = np.zeros((redo_calc, 2**qubits))
local_noiseless_static_solution = np.zeros((redo_calc, 2**qubits))

global_noise_dynamic_solution = np.zeros((redo_calc, 2**qubits))
global_noiseless_dynamic_solution = np.zeros((redo_calc, 2**qubits))
global_noise_static_solution = np.zeros((redo_calc, 2**qubits))
global_noiseless_static_solution = np.zeros((redo_calc, 2**qubits))

correct_solutions = np.zeros((redo_calc, 2**qubits))


device = Device(DeviceType.QISKIT_AER, qubits=qubits)

param_shape = (qubits + depth*(qubits+qubits -2),)

for i in tqdm(range(redo_calc)):
        a = poissonMatrix(2**(qubits//2)).todense()
        classical_solution = np.linalg.solve(a, b)
        normalized_classical_solution = np.square(classical_solution / np.linalg.norm(classical_solution))
        normalized_classical_solution /= np.linalg.norm(normalized_classical_solution)

        correct_solutions[i] = np.abs(normalized_classical_solution)

        ###################################
        #####    DYNAMIC NOISELESS    #####
        ###################################
        lse_local = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=True, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                silent=True)
        
        lse_global = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=False, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                silent=True)

        solution_local, param_local, it_count_local = lse_local.solve()
        solution_global, param_global, it_count_global = lse_global.solve()
        solution_local /= np.linalg.norm(solution_local)
        solution_global /= np.linalg.norm(solution_global)

        local_noiseless_dynamic_solution[i] = solution_local
        global_noiseless_dynamic_solution[i] = solution_global

        local_noiseless_dynamic_trc[i] = TRC_ADA(it_count_local)
        global_noiseless_dynamic_trc[i] = TRC_ADA(it_count_global)

        local_noiseless_dynamic_error[i] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
        global_noiseless_dynamic_error[i] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

        ###################################
        #####    DYNAMIC NOISEFULL    #####
        ###################################
        lse_local = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=True, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                device=device,
                silent=True)
        
        lse_global = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=False, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                device=device,
                silent=True)

        solution_local, param_local, it_count_local = lse_local.solve()
        solution_global, param_global, it_count_global = lse_global.solve()
        solution_local /= np.linalg.norm(solution_local)
        solution_global /= np.linalg.norm(solution_global)

        local_noise_dynamic_solution[i] = solution_local
        global_noise_dynamic_solution[i] = solution_global

        local_noise_dynamic_trc[i] = TRC_ADA(it_count_local)
        global_noise_dynamic_trc[i] = TRC_ADA(it_count_global)

        local_noise_dynamic_error[i] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
        global_noise_dynamic_error[i] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

        ###################################
        #####    STATIC  NOISELESS    #####
        ###################################
        lse_local = DeviceVarLSESolver(a,
                b, 
                method='direct',
                ansatz=fixed_layered_ansatz,
                weights=param_shape,
                local=True, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                silent=True)
        
        lse_global = DeviceVarLSESolver(a,
                b, 
                method='direct',
                ansatz=fixed_layered_ansatz,
                weights=param_shape,
                local=False, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                silent=True)

        solution_local, param_local, it_count_local = lse_local.solve()
        solution_global, param_global, it_count_global = lse_global.solve()
        solution_local /= np.linalg.norm(solution_local)
        solution_global /= np.linalg.norm(solution_global)

        local_noiseless_static_solution[i] = solution_local
        global_noiseless_static_solution[i] = solution_global

        local_noiseless_static_trc[i] = TRC_ASA(it_count_local)
        global_noiseless_static_trc[i] = TRC_ASA(it_count_global)

        local_noiseless_static_error[i] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
        global_noiseless_static_error[i] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

        ###################################
        #####    STATIC  NOISEFULL    #####
        ###################################
        lse_local = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=True, 
                ansatz=fixed_layered_ansatz,
                weights=param_shape,
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                device=device,
                silent=True)
        
        lse_global = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=False, 
                ansatz=fixed_layered_ansatz,
                weights=param_shape,
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                device=device,
                silent=True)

        solution_local, param_local, it_count_local = lse_local.solve()
        solution_global, param_global, it_count_global = lse_global.solve()
        solution_local /= np.linalg.norm(solution_local)
        solution_global /= np.linalg.norm(solution_global)

        local_noise_static_solution[i] = solution_local
        global_noise_static_solution[i] = solution_global

        local_noise_static_trc[i] = TRC_ASA(it_count_local)
        global_noise_static_trc[i] = TRC_ASA(it_count_global)

        local_noise_static_error[i] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
        global_noise_static_error[i] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:33<00:00, 16.55s/it]


## __Save vals__

In [7]:
## Save the vals
filename = f"data/result_poisson_test_1"
saved_dict = {
    # Noise dynamic
    "local_noise_dynamic_trc" : local_noise_dynamic_trc,
    "local_noise_dynamic_error" : local_noise_dynamic_error,
    "local_noise_dynamic_solution" : local_noise_dynamic_solution,
    "global_noise_dynamic_trc" : global_noise_dynamic_trc,
    "global_noise_dynamic_error" : global_noise_dynamic_error,
    "global_noise_dynamic_solution" : global_noise_dynamic_solution,

    # Noiseless dynamic
    "local_noiseless_dynamic_trc" : local_noiseless_dynamic_trc,
    "local_noiseless_dynamic_error" : local_noiseless_dynamic_error,
    "local_noiseless_dynamic_solution" : local_noiseless_dynamic_solution,
    "global_noiseless_dynamic_trc" : global_noiseless_dynamic_trc,
    "global_noiseless_dynamic_error" : global_noiseless_dynamic_error,
    "global_noiseless_dynamic_solution" : global_noiseless_dynamic_solution,

    # Noise static
    "local_noise_static_trc" : local_noise_static_trc,
    "local_noise_static_error" : local_noise_static_error,
    "local_noise_static_solution" : local_noise_static_solution,
    "global_noise_static_trc" : global_noise_static_trc,
    "global_noise_static_error" : global_noise_static_error,
    "global_noise_static_solution" : global_noise_static_solution,

    # Noiseless static
    "local_noiseless_static_trc" : local_noiseless_static_trc,
    "local_noiseless_static_error" : local_noiseless_static_error,
    "local_noiseless_static_solution" : local_noiseless_static_solution,
    "global_noiseless_static_trc" : global_noiseless_static_trc,
    "global_noiseless_static_error" : global_noiseless_static_error,
    "global_noiseless_static_solution" : global_noiseless_static_solution,

    "normalized_classical_solution" : normalized_classical_solution
}
np.savez(filename, **saved_dict)
print(f"Saved {filename}")

Saved data/result_poisson_test_1


In [ ]:
data = np.load(f"{filename}.npz")

local_noise_dynamic_error = data["local_noise_dynamic_error"]
print(local_noise_dynamic_error)

[0.77176801 0.86055383]
